In [ ]:
import yfinance as yf
from fastapi import FastAPI
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, MetaData, select
import nest_asyncio
from datetime import datetime
from typing import Optional
import pandas as pd
from fastapi.responses import HTMLResponse
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import json
from plotly.offline import plot


from fastapi.responses import FileResponse


nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Database setup
engine = create_engine("sqlite:///index_data.db")
metadata = MetaData()

index_data = Table("index_data", metadata,
                   Column("id", Integer, primary_key=True),
                   Column("symbol", String),
                   Column("weight", Float),
                   Column("value", Float),
                   Column("date", String)
                   )

metadata.create_all(engine)

# Constants
TECH_STOCKS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']

def fetch_stock_data(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    stock_data = stock.history(start=start_date, end=end_date)
    return stock_data


def store_stock_data(symbol, stock_data):
    with engine.begin() as connection:
        for date, data in stock_data.iterrows():
            connection.execute(index_data.insert().values(symbol=symbol, weight=data["Close"], value=data["Volume"], date=date.strftime("%Y-%m-%d")))


def calculate_value_weighted_index():
    with engine.begin() as connection:
        query = select([index_data.c.symbol, index_data.c.weight, index_data.c.value, index_data.c.date]).order_by(index_data.c.date)
        stock_data = connection.execute(query).fetchall()

    # Calculate value-weighted index with monthly rebalancing (simplified)
    index = {}
    last_month = None
    for symbol, weight, value, date in stock_data:
        dt = datetime.strptime(date, "%Y-%m-%d")
        month_key = (dt.year, dt.month)

        if last_month != month_key:
            if last_month is not None:
                index[last_month] = index_value
            index_value = 0
            last_month = month_key

        index_value += weight * value

    index[last_month] = index_value
    return index


@app.on_event("startup")
async def startup_event():
    # Fetch stock data and store it in the database
    start_date = "2010-01-01"
    end_date = "2022-12-31"
    for symbol in TECH_STOCKS:
        stock_data = fetch_stock_data(symbol, start_date, end_date)
        store_stock_data(symbol, stock_data)

    # Calculate value-weighted index
    global value_weighted_index
    value_weighted_index = calculate_value_weighted_index()


def generate_plot(stock_data: pd.DataFrame, start_date: str, end_date: str) -> str:
    symbols = stock_data.index.get_level_values(0).unique()

    fig = go.Figure()

    for symbol in symbols:
        symbol_data = stock_data.loc[symbol]  # Select data for the specific symbol
        symbol_data = symbol_data.reset_index().drop_duplicates(subset='date', keep='first').set_index('date')

        closest_start_date = symbol_data.index.get_indexer([start_date], method='nearest')[0]
        closest_end_date = symbol_data.index.get_indexer([end_date], method='nearest')[0]

        symbol_data = symbol_data.iloc[closest_start_date:closest_end_date + 1]
        fig.add_trace(go.Scatter(x=symbol_data.index, y=symbol_data['weight'], name=symbol))

    fig.update_layout(title='Stock Prices', xaxis_title='Date', yaxis_title='Price')

    plot_html = plot(fig, output_type='div', include_plotlyjs=False)
    return plot_html




@app.get("/")
async def read_root():
    return FileResponse("index.html")

@app.get("/index/{date}", response_class=HTMLResponse)
def get_index_value(date: str, invested: Optional[float] = None, end_date: Optional[str] = None):
    dt = datetime.strptime(date, "%Y-%m-%d")
    month_key = (dt.year, dt.month)

    if month_key not in value_weighted_index:
        return {"error": "Index data not available for the specified date"}

    response = {"date": date, "index_value": value_weighted_index[month_key]}

    if invested is not None and end_date is not None:
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")
        end_month_key = (end_dt.year, end_dt.month)

        if end_month_key not in value_weighted_index:
            return {"error": "Index data not available for the specified end date"}

        initial_value = value_weighted_index[month_key]
        final_value = value_weighted_index[end_month_key]
        profit_loss = (final_value - initial_value) * (invested / initial_value)
        response["invested"] = invested
        response["end_date"] = end_date
        response["profit_loss"] = profit_loss

    with engine.begin() as connection:
        query = select([index_data.c.symbol, index_data.c.weight, index_data.c.value, index_data.c.date]).order_by(index_data.c.date)
        stock_data = connection.execute(query).fetchall()

    stock_data = pd.DataFrame(stock_data, columns=['symbol', 'weight', 'value', 'date'])
    stock_data['date'] = pd.to_datetime(stock_data['date'])
    stock_data.set_index(['symbol', 'date'], inplace=True)

    start_date = datetime.strptime(date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    plot_html = generate_plot(stock_data, start_date, end_date)

    return f"""
    <html>
        <head>
            <title>Index Value</title>
        </head>
        <body>
            <h1>Index Value</h1>
            <p>Date: {date}</p>
            <p>Index value: {response['index_value']}</p>
            {f"<p>Invested: ${response['invested']}</p>" if invested is not None else ""}
            {f"<p>End date: {response['end_date']}</p>" if end_date is not None else ""}
            {f"<p>Profit/Loss: ${response['profit_loss']:.2f}</p>" if invested is not None and end_date is not None else ""}
            {plot_html}
        </body>
    </html>
    """

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [7414]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     38.42.50.182:0 - "GET / HTTP/1.1" 200 OK
INFO:     38.42.50.182:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     38.42.50.182:0 - "GET /index/2020-06-01?invested=1000&end_date=2020-10-01 HTTP/1.1" 200 OK
INFO:     38.42.50.182:0 - "GET / HTTP/1.1" 200 OK
INFO:     38.42.50.182:0 - "GET /index/2020-01-01?invested=1000&end_date=2020-12-01 HTTP/1.1" 200 OK
INFO:     38.42.50.182:0 - "GET /index/2020-01-01?invested=1000&end_date=2022-12-01 HTTP/1.1" 200 OK
INFO:     38.42.50.182:0 - "GET /index/2010-01-01?invested=1000&end_date=2022-12-01 HTTP/1.1" 200 OK
INFO:     38.42.50.182:0 - "GET /index/2010-01-01?invested=1000&end_date=2010-12-01 HTTP/1.1" 200 OK
INFO:     38.42.50.182:0 - "GET /index/2020-01-01?invested=1000&end_date=2020-12-01 HTTP/1.1" 200 OK


In [12]:
stock_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-05-18 00:00:00-04:00,42.049999,45.000000,38.000000,38.230000,573576400,0.0,0.0
2012-05-21 00:00:00-04:00,36.529999,36.660000,33.000000,34.029999,168192700,0.0,0.0
2012-05-22 00:00:00-04:00,32.610001,33.590000,30.940001,31.000000,101786600,0.0,0.0
2012-05-23 00:00:00-04:00,31.370001,32.500000,31.360001,32.000000,73600000,0.0,0.0
2012-05-24 00:00:00-04:00,32.950001,33.209999,31.770000,33.029999,50237200,0.0,0.0
...,...,...,...,...,...,...,...
2022-12-23 00:00:00-05:00,116.029999,118.180000,115.540001,118.040001,17796600,0.0,0.0
2022-12-27 00:00:00-05:00,117.930000,118.599998,116.050003,116.879997,21392300,0.0,0.0
2022-12-28 00:00:00-05:00,116.250000,118.150002,115.510002,115.620003,19612500,0.0,0.0
